# 2.1 Arrays

Great, so we already have a gradient descent for 1D functions. The next step is to extend it to functions of more than one variable; for example consider the function

$$ f(x,y) = x^2 + 2y^2 $$

It is obvious that with the current implementation we could not minimize it. For addressing this, we have to introduce *arrays*.

Arrays are pretty similar to those in Matlab. Let's start by defining a vector:

In [ ]:
# Define a vector
v = [1, 2, 3, 4]

In `Array{Int64,1}`, as you may probably guess, `Int64` tells the type of the elements of the array, while `1` is the number of dimensions. Julia is *column-mayor*, i.e. arrays are stored by columns in memory, so vectors are *column-vectors*. On the other hand, if we define a row vector, we obtain already a 2 dimensional array:

In [ ]:
v = [1 2 3 4]

In [ ]:
# Define a matrix
A = [1 2; 3 4]

In [ ]:
# or equivalently
B = [1 2;
     3 4]

A == B

Accesing elements is pretty straightforward

In [ ]:
println(v[1])
println(A[1, 2])
println(A[end, end])

Julia sees one- and two-dimensional arrays as vectors and matrices, so their multiplication is the matrix multiplication. For pointwise operation one must use `.*`:

In [ ]:
A * [1,1]

In [ ]:
A .* A

This *a point implies elementwise operation* concept extends to all functions, even user defined! This is extremely useful:

In [ ]:
g(x) = x^2 + 1
g.(A) 

In [ ]:
sqrt.(A)

In this last example, note that how Julia has changed the *type* of the Array from `Int64` to `Float64`. It is usually convenient to define the type of your arrays from the beginning; thus we can do:

In [ ]:
A = [1. 2; 3 4] # the point after the one indicates that its type is float, forcing the rest ot types

Of course there are also methods for defining arrays of zeros, ones, uninitialized arrays...

In [ ]:
zeros(Float64, 2, 2), ones(Float64, 2, 2), Array{Float64, 2}(undef, 2, 2)

Arrays can also be created via *comprehension*: check the folowing:

In [ ]:
a = [g(i) for i in 1:4]

This also works for nested loops! In the following example, check which loops correspond to rows and which to columns:

In [ ]:
h(x,y) = x-y
x = 0:2:10
y = x
Z = [h(xi,yj) for xi in x, yj in y]

This is also the Julian way of computing the output of a 2D-valued function when you want to do a contour/surface plot; while in Python you would do something like:

```python
x = np.linspace(0,10,6)
X, Y = np.meshgrid(x,x)
Z = h(X,Y)
```

In Julia you just don't create those (potentially big) `X` and `Y`, but just compute `Z` directly, like in the example above.

## 2.1.1 Important differences with respect to Matlab

There are of course plenty of differences between Julia and Matlab, but since arrays are so vital in every scientific computing code, we may well state here some of the most important:

* Julia does not automatically grow arrays in an assignment statement, i. e., if we have a vector `a` of length 4 we cannot expand it by doing `a[5] = 1`; we would need to write `a = [a; 1]`.
* Maybe the most important of all: Julia arrays are **not** copied when assigned to another variable. After `a = b`, changing elements of `b` will modify `a` as well:

In [ ]:
a = [1,2,3]
b = a
a[1] = 0
b

An extensive list can be found here: https://docs.julialang.org/en/v1/manual/noteworthy-differences/#Noteworthy-differences-from-MATLAB-1

## 2.1.2 Important differences with respect to Python

Some other differences with respect to Python are:

* Julia has 1-based indexing, instead of 0-based indexing as in Python.
* Julia's slice indexing includes the last element, unlike in Python. `a[2:3]` in Julia is `a[1:3]` in Python

In [ ]:
a = [1, 2, 3]
a[2:3] # this would be like 1:3 in Python

An extensive set of differences can be found at https://docs.julialang.org/en/v1/manual/noteworthy-differences/#Noteworthy-differences-from-Python-1

## 2.1.3 Arrays as lists and concatenation

Thinking back about our gradient descent, apart from extending it to multidimensional functions, it would also be nice to have a history of the iterations. For this we need some kind of list to store them in. 

If you come from Python it may surprise you to know there are no lists in Julia. Instead, list operations are done using arrays. For example, pushing an element to the end of an array can be done as:

In [ ]:
c = [5, 1]
push!(c, 3)
c

You may have noticed that `!` at the end of `push!`. In Julia there is the convention of naming the functions that modify its arguments ending with a `!`. Thus, we have two `sort`s: the first doesn't change the array `C` itself:

In [ ]:
sort(c), c

While `sort!` *does* modify it:

In [ ]:
sort!(c), c

Anyway, some users will feel conforted to have also the following resources for expanding arrays:

In [ ]:
c = [c;2]

In [ ]:
D = [1 2]; D = [D 3]

Note that we use a semicolon to concatenate vertically and a space to concatenate horizontally. Using a comma instead returns an array of type `Any` containing the given elements:

In [ ]:
D = [1 2]; D = [D, 3]

# 2.2 Packages

As you may remember, our stopping criterion in `gradient_descent` involved checking `abs(Df(x))`. Since now `Df(x)` will be an array, we need a way to compute its norm. We could of course code it ourselves, but Julia comes with a rich library environment that has a lot of this work already done. 

In particular, the function `norm` comes in the package `LinearAlgebra.jl`. If you are running this notebook in Binder the package is already installed; if not you can install it by uncommenting and running the following cell:

In [ ]:
# using Pkg
# Pkg.add("LinearAlgebra")

In [ ]:
using LinearAlgebra # since we only need the function `norm` we could also do `using LinearAlgebra: norm`

a = [1, 1]
norm(a)

Great! We are again prepared for coding. Let us first define the function we want to minimize and its derivate. Let's make it grow faster in the $y$ than in the $x$ direction to check it our algorithm works properly:

![surface plot](surface_plot.png)

In [ ]:
g(x) = x[1]^2 + 2*x[2]^2
Dg(x) = 2*[x[1], 2*x[2]]

#### Exercise 2

Build the function $g(x) = x_1^2 + 2x_2^2$ and its gradient using only matrix and scalar-matrix operations (Hint: the transpose of the vector `x` is `x'`).

In [ ]:
# Your solution goes here

#### Exercise 3
Bring the gradient descent of the first notebook into the multidimensional realm, and make it output the _history_ of `x`s and `f(x)`s (Hint: consider using the concatenate functions that were explained above.)

In [ ]:
""" Gradient descent for multidimensional functions Stops when the gradient is smaller than `TOL`, 
    or when the maximum number of iterations `maxiter` has been reached"""
function gradient_descent(f, Df, x; α = 0.1, TOL = 1e-10, maxiter = 1000, verbose = false)
    xn = [x...]
    fn = [f(x)]
    
    # Your code goes here
    
    return(xn, fn)
end

In [ ]:
# Let's test it
xn, fn = gradient_descent(g, Dg, [1., 1], α = 0.01, verbose = true); # We can add a semicolon to mute the output

Well, enough of sawing raw data. In the next notebook we will learn how to visualize it!